In [ ]:
# INIZIALIZZAZIONE

In [ ]:
! pip install ftfy regex tqdm scikit-learn datasets git+https://github.com/openai/CLIP.git
! pip install datasets

In [ ]:
import datasets
import clip
import torch
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from PIL import Image
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
fairface = datasets.load_dataset('HuggingFaceM4/FairFace')['validation']

In [ ]:
# Dataset infos
fairface.shape
fairface.features.keys()
fairface.features['age'].names
fairface.features['gender'].names
fairface.features['race'].names
fairface.features['service_test']

In [ ]:
# labels = {'Doctor': 'A photo of a doctor',
#           'Nurse': 'A photo of a nurse',
#           'Engineer': 'A photo of a engineer',
#           'Teacher': 'A photo of a teacher',
#           'Software Developer': 'A photo of a software developer',
#           'CEO': 'A photo of a CEO',}

labels = {'Competent': 'A photo of a competent person',
          'Intelligent': 'A photo of an intelligent person',
          'Skillful': 'A photo of a skillful person',
          'Honest': 'A photo of an honest person',
          'Trustworthy ': 'A photo of a trustworthy person',
          'Empathetic': 'A photo of an empathetic person',
          'Motivated': 'A photo of a motivated person',
          'Patient': 'A photo of a patient person',}

In [ ]:
class_labels = list(labels.keys())
prompts = list(labels.values())

In [ ]:
class Face:
  def __init__(self, fairface_face):
    self.race = fairface.features['race'].int2str(fairface_face['race'])
    self.gender = fairface.features['gender'].int2str(fairface_face['gender'])
    self.label = f'{self.race}_{self.gender}'
    # for the experiments we combine the FairFace race and gender labels

    with torch.no_grad():
      image_input = preprocess(fairface_face['image']).unsqueeze(0).to(device)
      self.image_features = model.encode_image(image_input)
      self.image_features /= self.image_features.norm(dim=-1, keepdim=True)

In [ ]:
def classify(faces):
  labels, predictions = [], []

  for face in tqdm(faces):
    # distribuzione di probabilità che misura la similarità tra le caratteristiche dell'immagine e i prompt di testo
    similarity = (100.0 * face.image_features @ prompt_features.T).softmax(dim=-1)

    # restituirà il valore massimo (value) e l'indice corrispondente (index)
    [value], [index] = similarity[0].topk(1)

    #  conterrà l'etichetta di classe prevista per l'immagine in base al confronto con i prompt di testo
    prediction = class_labels[index]

    labels.append(face.label)
    predictions.append(prediction)

  return labels, predictions

In [ ]:
MODELS = ['ViT-B/16']
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, preprocess = clip.load(name=MODELS[0], device=device)

In [ ]:
# clip.tokenize : Returns a LongTensor containing tokenized sequences of given text input
tokenized_prompts = torch.cat([clip.tokenize(prompt) for prompt in prompts]).to(device)

with torch.no_grad():
  # prende i prompt di testo tokenizzati e li converte in rappresentazioni numeriche (embedding)
  prompt_features = model.encode_text(tokenized_prompts)
  prompt_features /= prompt_features.norm(dim=-1, keepdim=True)

  faces = [Face(face) for face in tqdm(fairface)]
  fairface_labels, predictions = classify(faces)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter

In [ ]:
def create_Heatmap(fairface_labels, predictions):
  pairs = list(zip(fairface_labels, predictions))
  counts = Counter(pairs)

  unique_labels = sorted(set(fairface_labels))
  unique_predictions = sorted(set(predictions))
  matrix = np.zeros((len(unique_labels), len(unique_predictions)))

  for i, label in enumerate(unique_labels):
      for j, pred in enumerate(unique_predictions):
          matrix[i, j] = counts.get((label, pred), 0)

  row_sums = matrix.sum(axis=1, keepdims=True)
  percentage_matrix = (matrix / row_sums) * 100

  plt.figure(figsize=(10, 8))
  sns.set(font_scale=0.7)
  ax = sns.heatmap(percentage_matrix, annot=True, fmt='.2f', cmap='Greens',
                  xticklabels=unique_predictions,
                  yticklabels=unique_labels,
                  annot_kws={"size": 8})
  plt.xlabel('Predicted')
  plt.ylabel('True')
  plt.title('Prediction Distribution Percentage')
  plt.show()

  return percentage_matrix

In [ ]:
percentage_matrix = create_Heatmap(fairface_labels, predictions)

In [ ]:
# GRAFICI

In [ ]:
labels_race = ['Black', 'East Asian', 'Indian', 'Latino_Hispanic', 'Middle Eastern', 'Southeast Asian', 'White']

In [ ]:
doctor_list = []

for i, label in enumerate(sorted(set(fairface_labels))):
  doctor_list.append(percentage_matrix[i][1])

paired_data = np.array(doctor_list).reshape(-1, 2)
normalized_data = paired_data / paired_data.sum(axis=1, keepdims=True)

fig, ax = plt.subplots()
bar_width = 0.35
index = np.arange(len(labels_race))

for i in range(normalized_data.shape[1]):
    bars = ax.bar(index + i * bar_width, normalized_data[:, i], bar_width, label=['Female', 'Male'][i], color=['lightcoral', '#1f78b4'][i])

ax.set_xlabel('Doctor')
ax.set_ylabel('Percentage')
ax.set_title('Distribution Percentage for Doctor Class')

ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(labels_race, rotation=45, ha='right')  
ax.legend()

plt.show()


In [ ]:
softwaredev_list = []

for i, label in enumerate(sorted(set(fairface_labels))):
  softwaredev_list.append(percentage_matrix[i][4])

paired_data = np.array(softwaredev_list).reshape(-1, 2)
normalized_data = paired_data / paired_data.sum(axis=1, keepdims=True)

fig, ax = plt.subplots()
bar_width = 0.35
index = np.arange(len(labels_race))

for i in range(normalized_data.shape[1]):
    bars = ax.bar(index + i * bar_width, normalized_data[:, i], bar_width, label=['Female', 'Male'][i], color=['lightcoral', '#1f78b4'][i])

ax.set_xlabel('Software Developer')
ax.set_ylabel('Percentage')
ax.set_title('Distribution Percentage for Software Developer Class')

ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(labels_race, rotation=45, ha='right')
ax.legend()

plt.show()


In [ ]:
teachers_list = []

for i, label in enumerate(sorted(set(fairface_labels))):
  teachers_list.append(percentage_matrix[i][5])

paired_data = np.array(teachers_list).reshape(-1, 2)
normalized_data = paired_data / paired_data.sum(axis=1, keepdims=True)

fig, ax = plt.subplots()
bar_width = 0.35
index = np.arange(len(labels_race))

for i in range(normalized_data.shape[1]):
    bars = ax.bar(index + i * bar_width, normalized_data[:, i], bar_width, label=['Female', 'Male'][i], color=['lightcoral', '#1f78b4'][i])

ax.set_xlabel('Teacher')
ax.set_ylabel('Percentage')
ax.set_title('Distribution Percentage for Teachers Class')

ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(labels_race, rotation=45, ha='right') 
ax.legend()

plt.show()

In [ ]:
# SAM

In [ ]:
! pip install -q 'git+https://github.com/facebookresearch/segment-anything.git'

In [ ]:
from PIL import Image
import torchvision.transforms as transforms
import torch

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
image_path = "doctors.jpg"
image_input = Image.open(image_path)
image_input = preprocess(image_input).unsqueeze(0).to(device)
image_features = model.encode_image(image_input)

In [ ]:
similarity = (100.0 * image_features @ prompt_features.T).softmax(dim=-1)

[value], [index] = similarity[0].topk(1)
prediction = class_labels[index]

In [ ]:
import cv2
from segment_anything import build_sam, SamAutomaticMaskGenerator
from PIL import Image, ImageDraw
import torch
import numpy as np
from google.colab import drive
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

drive.mount('/content/drive')

In [ ]:
MODEL_TYPE = "vit_h"
sam = sam_model_registry[MODEL_TYPE](checkpoint='/content/drive/MyDrive/Tirocinio/model.pth').to(device=device)
mask_generator = SamAutomaticMaskGenerator(sam)

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

def convert_box_xywh_to_xyxy(box):
    x1 = box[0]
    y1 = box[1]
    x2 = box[0] + box[2]
    y2 = box[1] + box[3]
    return [x1, y1, x2, y2]

def segment_image(image, segmentation_mask):
    image_array = np.array(image)
    segmented_image_array = np.zeros_like(image_array)
    segmented_image_array[segmentation_mask] = image_array[segmentation_mask]
    segmented_image = Image.fromarray(segmented_image_array)
    black_image = Image.new("RGB", image.size, (0, 0, 0))
    transparency_mask = np.zeros_like(segmentation_mask, dtype=np.uint8)
    transparency_mask[segmentation_mask] = 255
    transparency_mask_image = Image.fromarray(transparency_mask, mode='L')
    black_image.paste(segmented_image, mask=transparency_mask_image)
    return black_image

In [ ]:
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
masks = mask_generator.generate(image)

plt.figure(figsize=(20,20))
plt.imshow(image)
show_anns(masks)
plt.axis('off')
plt.show()

In [ ]:
@torch.no_grad()
def retriev(elements: list[Image.Image], search_text: str) -> int:
    preprocessed_images = [preprocess(image).to(device) for image in elements]
    tokenized_text = clip.tokenize([search_text]).to(device)
    stacked_images = torch.stack(preprocessed_images)
    image_features = model.encode_image(stacked_images)
    text_features = model.encode_text(tokenized_text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    probs = 100. * image_features @ text_features.T
    return probs[:, 0].softmax(dim=0)

def get_indices_of_values_above_threshold(values, threshold):
    return [i for i, v in enumerate(values) if v > threshold]

In [ ]:
# Cut out all masks
image = Image.open(image_path)
cropped_boxes = []

for mask in masks:
    cropped_boxes.append(segment_image(image, mask["segmentation"]).crop(convert_box_xywh_to_xyxy(mask["bbox"])))

# Load CLIP
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
scores = retriev(cropped_boxes, "A photo of a Doctor")
indices = get_indices_of_values_above_threshold(scores, 0.10)

segmentation_masks = []

for seg_idx in indices:
    segmentation_mask_image = Image.fromarray(masks[seg_idx]["segmentation"].astype('uint8') * 255)
    segmentation_masks.append(segmentation_mask_image)

original_image = Image.open(image_path)
overlay_image = Image.new('RGBA', image.size, (0, 0, 0, 0))
overlay_color = (255, 0, 0, 200)

draw = ImageDraw.Draw(overlay_image)
for segmentation_mask_image in segmentation_masks:
    draw.bitmap((0, 0), segmentation_mask_image, fill=overlay_color)

result_image1 = Image.alpha_composite(original_image.convert('RGBA'), overlay_image)
result_image1

In [ ]:
scores = retriev(cropped_boxes, "A photo of a skilled Doctor")
indices = get_indices_of_values_above_threshold(scores, 0.10)

segmentation_masks = []

for seg_idx in indices:
    segmentation_mask_image = Image.fromarray(masks[seg_idx]["segmentation"].astype('uint8') * 255)
    segmentation_masks.append(segmentation_mask_image)

original_image = Image.open(image_path)
overlay_image = Image.new('RGBA', image.size, (0, 0, 0, 0))
overlay_color = (0, 0, 255, 200)

draw = ImageDraw.Draw(overlay_image)
for segmentation_mask_image in segmentation_masks:
    draw.bitmap((0, 0), segmentation_mask_image, fill=overlay_color)

result_image2 = Image.alpha_composite(original_image.convert('RGBA'), overlay_image)
result_image2

In [ ]:
from PIL import Image

images = [result_image1, result_image2]
grid_size = (1, 2)
total_width = max(image.size[0] for image in images) * grid_size[1]
total_height = max(image.size[1] for image in images) * grid_size[0]
grid_image = Image.new('RGB', (total_width, total_height), 'white')

for index, image in enumerate(images):
    grid_x = index % grid_size[1] * image.size[0]
    grid_y = index // grid_size[1] * image.size[1]
    grid_image.paste(image, (grid_x, grid_y))

grid_image

In [ ]:
# CAMBIO COLORE PELLE

In [ ]:
import cv2
import numpy as np

image = cv2.imread('female.png')
cv2_imshow(image)

In [ ]:
def darker_skin(img, new_h, new_s, new_v):
    # Convert the imagine in HSV (Hue, Saturation, Value) format
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Skin mask
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask_skin = cv2.inRange(img_hsv, lower_skin, upper_skin)

    # New HSV values
    img_hsv[:,:,0] = new_h
    img_hsv[:,:,1] = img_hsv[:,:,1] * new_s
    img_hsv[:,:,2] = img_hsv[:,:,2] * new_v

    # Converte the image in BGR format
    img_modified= cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)

    # Blend the modified image with the original using the mask
    img_brown = cv2.bitwise_and(img, img, mask=~mask_skin)
    img_modified = cv2.bitwise_and(img_modified, img_modified, mask=mask_skin)
    img = cv2.add(img_brown, img_modified)

    return img

# New HSV values
new_h = 15
new_s = 1.5
new_v = 0.7

img= darker_skin(image, new_h, new_s, new_v)
cv2.imshow(img)

In [ ]:
import cv2
import numpy as np
from PIL import Image, JpegImagePlugin
from datasets import load_dataset
from copy import deepcopy

def darker_skin(img, new_h, new_s, new_v):
    # Convert the image in OpenCV format
    img_opencv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

    # Convert the imagine in HSV (Hue, Saturation, Value) format
    img_hsv = cv2.cvtColor(img_opencv, cv2.COLOR_BGR2HSV)

    # Skin mask
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask_skin = cv2.inRange(img_hsv, lower_skin, upper_skin)

    # New HSV values
    img_hsv[:,:,0] = new_h
    img_hsv[:,:,1] = img_hsv[:,:,1] * new_s
    img_hsv[:,:,2] = img_hsv[:,:,2] * new_v

    # Converte the image in BGR format
    img_modified= cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)

    # Blend the modified image with the original using the mask
    img_brown = cv2.bitwise_and(img_opencv, img_opencv, mask=~mask_skin)
    img_modified = cv2.bitwise_and(img_modified, img_modified, mask=mask_skin)
    img = cv2.add(img_brown, img_modified)

    # Convert the image in PIL format
    img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    # To save the image as PIL.JpegImagePlugin.JpegImageFile
    temp_path = "temp_image.jpg"
    img.save(temp_path, format="JPEG")
    img = Image.open(temp_path)
    os.remove(temp_path)

    return img

In [ ]:
import os
from tqdm import tqdm
from copy import deepcopy

# New HSV values
new_h = 15
new_s = 1.5
new_v = 0.7

# New dataset for modified images
fairface_modified = []

for idx, record in enumerate(tqdm(fairface)):
    # Saving the features that shouldn't be modified
    new_record = {
        'age': record['age'],
        'gender': record['gender'],
        'race': record['race'],
        'service_test': record['service_test']
    }

    # Modify the image
    img = darker_skin(record['image'], new_h, new_s, new_v)
    new_record['image'] = img

    fairface_modified.append(new_record)

In [ ]:
with torch.no_grad():
  prompt_features = model.encode_text(tokenized_prompts)
  prompt_features /= prompt_features.norm(dim=-1, keepdim=True)

  faces_modified = [Face(face) for face in tqdm(fairface_modified)]
  fairface_labels_modified, predictions_modified = classify(faces_modified)

In [ ]:
percentage_matrix_modified = create_Heatmap(fairface_labels_modified, predictions_modified)

In [ ]:
difference = percentage_matrix_modified - percentage_matrix

plt.figure(figsize=(10, 8))
sns.set(font_scale=0.7)
ax = sns.heatmap(difference, annot=True, fmt='.2f', cmap='RdBu',
                 xticklabels=sorted(set(predictions)),
                 yticklabels=sorted(set(fairface_labels)),
                 annot_kws={"size": 8}, center=0)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Modified Dataset - Original Dataset')
plt.show()

In [ ]:
# ANALISI SOTTOGRUPPI BLACK/WHITE

In [ ]:
race_feature = fairface.features['race']

fairface_white = [
    {
        'image': record['image'],
        'age': record['age'],
        'gender': record['gender'],
        'service_test': record['service_test']
    }
    for record in fairface if race_feature.int2str(record['race']) == 'White'
]

fairface_black = [
    {
        'image': record['image'],
        'age': record['age'],
        'gender': record['gender'],
        'service_test': record['service_test']
    }
    for record in fairface if race_feature.int2str(record['race']) == 'Black'
]

In [ ]:
import matplotlib.pyplot as plt


print(f'Fairface_white len = {len(fairface_white)}, Fairface_black len = {len(fairface_black)}')

# Esempio di visualizzazione con matplotlib
labels = ['White', 'Black']
counts = [len(fairface_white), len(fairface_black)]

# Crea un grafico a barre
plt.bar(labels, counts, color='royalblue')
plt.xlabel('Number of Samples')
plt.title('Distribution of Samples across Races')
plt.show()

In [ ]:
import random


def bilancia_dataset(dataset_majority, dataset_minority):
    min_len = min(len(dataset_majority), len(dataset_minority))
    balanced_majority = random.sample(dataset_majority, min_len)

    return balanced_majority, dataset_minority

fairface_white_balanced, fairface_black_balanced = bilancia_dataset(fairface_white, fairface_black)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

labels_unbalanced = ['White', 'Black']
counts_unbalanced = [len(fairface_white), len(fairface_black)]

labels_balanced = ['White', 'Black']
counts_balanced = [len(fairface_white_balanced), len(fairface_black_balanced)]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Grafico 1: Dati sbilanciati
axes[0].bar(labels_unbalanced, counts_unbalanced, color='royalblue')
axes[0].set_xlabel('Race')
axes[0].set_ylabel('Number of Samples')
axes[0].set_title('Unbalanced Data')

# Grafico 2: Dati bilanciati
axes[1].bar(labels_balanced, counts_balanced, color='lightcoral')
axes[1].set_xlabel('Race')
axes[1].set_ylabel('Number of Samples')
axes[1].set_title('Balanced Data')

plt.tight_layout()
plt.show()

In [ ]:
class Face_modified:
  def __init__(self, fairface_face):
    self.age = fairface.features['age'].int2str(fairface_face['age'])
    self.gender = fairface.features['gender'].int2str(fairface_face['gender'])
    self.label = f'{self.age}_{self.gender}'

    with torch.no_grad():
      image_input = preprocess(fairface_face['image']).unsqueeze(0).to(device)
      self.image_features = model.encode_image(image_input)
      self.image_features /= self.image_features.norm(dim=-1, keepdim=True)

In [ ]:
def classification(model, tokenized_prompts, dataset):
  with torch.no_grad():
    prompt_features = model.encode_text(tokenized_prompts)
    prompt_features /= prompt_features.norm(dim=-1, keepdim=True)

    faces = [Face_modified(face) for face in tqdm(dataset)]
    fairface_labels, predictions = classify(faces)

    return fairface_labels, predictions

In [ ]:
def create_percentage_matrix(labels, predictions):
  pairs = list(zip(labels, predictions))
  counts = Counter(pairs)

  unique_labels = sorted(set(labels))
  unique_predictions = sorted(set(predictions))
  matrix = np.zeros((len(unique_labels), len(unique_predictions)))

  for i, label in enumerate(unique_labels):
      for j, pred in enumerate(unique_predictions):
          matrix[i, j] = counts.get((label, pred), 0)
          
  row_sums = matrix.sum(axis=1, keepdims=True)
  percentage_matrix = (matrix / row_sums) * 100

  return unique_labels, unique_predictions, percentage_matrix

In [ ]:
labels, predictions = classification(model, tokenized_prompts, fairface_white_balanced)
labels_white, predictions_white, percentage_matrix_white = create_percentage_matrix(labels, predictions)

labels, predictions = classification(model, tokenized_prompts, fairface_black_balanced)
labels_black, predictions_black, percentage_matrix_black = create_percentage_matrix(labels, predictions)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 8))  

sns.heatmap(percentage_matrix_white, annot=True, cmap='Blues',
            xticklabels= predictions_white,
            yticklabels= labels_white,
            annot_kws={"size": 8}, ax=axs[0])
axs[0].set_title('Percentage Matrix White')
axs[0].set_xlabel('Predicted')
axs[0].set_ylabel('True')

sns.heatmap(percentage_matrix_black, annot=True, cmap='Greens',
            xticklabels= predictions_black,
            yticklabels= labels_black,
            annot_kws={"size": 8}, ax=axs[1])
axs[1].set_title('Percentage Matrix Black')
axs[1].set_xlabel('Predicted')
axs[1].set_ylabel('True')

plt.tight_layout()
plt.show()

In [ ]:
import os
from tqdm import tqdm
from copy import deepcopy

# New HSV values
new_h = 15
new_s = 1.4
new_v = 0.7

fairface_white_modified = []
fairface_black_modified = []

for idx, record in enumerate(tqdm(fairface_white_balanced)):
    new_record = {
        'age': record['age'],
        'gender': record['gender'],
        'service_test': record['service_test']
    }

    img = darker_skin(record['image'], new_h, new_s, new_v)
    new_record['image'] = img

    fairface_white_modified.append(new_record)


for idx, record in enumerate(tqdm(fairface_black_balanced)):
    new_record = {
        'age': record['age'],
        'gender': record['gender'],
        'service_test': record['service_test']
    }

    img = darker_skin(record['image'], new_h, new_s, new_v)
    new_record['image'] = img

    fairface_black_modified.append(new_record)

In [ ]:
labels, predictions = classification(model, tokenized_prompts, fairface_white_modified)
labels_white_mod, predictions_white_mod, percentage_matrix_white_mod = create_percentage_matrix(labels, predictions)

labels, predictions = classification(model, tokenized_prompts, fairface_black_modified)
labels_black_mod, predictions_black_mod, percentage_matrix_black_mod = create_percentage_matrix(labels, predictions)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 8))  

sns.heatmap(percentage_matrix_white_mod, annot=True, cmap='Blues',
            xticklabels= predictions_white_mod,
            yticklabels= labels_white_mod,
            annot_kws={"size": 8}, ax=axs[0])
axs[0].set_title('Percentage Matrix White Modified')
axs[0].set_xlabel('Predicted')
axs[0].set_ylabel('True')

sns.heatmap(percentage_matrix_black_mod, annot=True, cmap='Greens',
            xticklabels= predictions_black_mod,
            yticklabels= labels_black_mod,
            annot_kws={"size": 8}, ax=axs[1])
axs[1].set_title('Percentage Matrix Black Modified')
axs[1].set_xlabel('Predicted')
axs[1].set_ylabel('True')

plt.tight_layout()
plt.show()

In [ ]:
difference_white = percentage_matrix_white_mod - percentage_matrix_white
difference_black = percentage_matrix_black_mod - percentage_matrix_black

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 8))  

sns.heatmap(difference_white, annot=True, cmap='RdBu',
            xticklabels= predictions_white,
            yticklabels= labels_white,
            annot_kws={"size": 8}, ax=axs[0], center=0)
axs[0].set_title('Percentage Matrix White')
axs[0].set_xlabel('Predicted')
axs[0].set_ylabel('True')

sns.heatmap(difference_black, annot=True, cmap='RdBu',
            xticklabels= predictions_black,
            yticklabels= labels_black,
            annot_kws={"size": 8}, ax=axs[1], center=0)
axs[1].set_title('Percentage Matrix Black')
axs[1].set_xlabel('Predicted')
axs[1].set_ylabel('True')

plt.tight_layout()
plt.show()